In [1]:
import random
from multiprocessing import Pool, cpu_count
from typing import Dict, List, Set, Tuple

import networkx as nx
import numpy as np
import pandas as pd

from icm_diffusion import simulate_diffusion_ICM
from src.graph import (
    create_polarized_graph,
    graph_loader,
    random_color_graph,
    spectral_bipartition_coloring,
)
from src.seed import seed_degree, seed_random, seed_mia, seed_polarized


In [2]:
from edge_addition import (
    edge_addition_custom,
    edge_addition_adamic_adar,
    edge_addition_preferential_attachment,
    edge_addition_jaccard,
    edge_addition_degree,
    edge_addition_topk,
    edge_addition_prob,
    edge_addition_kkt,
    edge_addition_random,
)

In [3]:
# Function to evaluate and compare the graph modifications
def evaluate_graph_modifications(G, seeds, k, max_iter, budget):
    # Simulate diffusion on the original graph
    count, count_std, color_count, color_count_std = simulate_diffusion_ICM(G, seeds, max_iter)
    
    # Results for the original graph
    original_results = pd.DataFrame({
        'Metric': ['Count', 'Count Standard Deviation', 'Color Count', 'Color Count Standard Deviation'],
        'Original Graph': [round(count, 3), round(count_std, 3), round(color_count, 3), round(color_count_std, 3)]
    })

    # Get the number of nodes and edges for all graphs
    graph_info = {
        'Metric': ['Number of Nodes', 'Number of Edges'],
        'Original Graph': [G.number_of_nodes(), G.number_of_edges()],
    }
    #'Adamic Adar': edge_addition_adamic_adar,
    # Define a list of modification functions
    modification_functions = {
        
        'PrefAtt': edge_addition_preferential_attachment,
        'Jaccard': edge_addition_jaccard,
        'Degree': edge_addition_degree,
        'TopK': edge_addition_topk,
        'Prob': edge_addition_prob,
        'KKT': edge_addition_kkt,
        'Random': edge_addition_random,
        'Custom': edge_addition_custom
    }

    combined_results = original_results.copy()

    # Evaluate each graph modification
    for method_name, mod_func in modification_functions.items():
        modified_graph = mod_func(G, seeds, k, budget)
        count, count_std, color_count, color_count_std = simulate_diffusion_ICM(modified_graph, seeds, max_iter)

        adapted_results = pd.DataFrame({
            'Metric': ['Count', 'Count Standard Deviation', 'Color Count', 'Color Count Standard Deviation'],
            f'Adapted Graph {method_name}': [round(count, 3), round(count_std, 3), round(color_count, 3), round(color_count_std, 3)]
        })

        combined_results = pd.merge(combined_results, adapted_results, on='Metric')
        graph_info[f'Adapted Graph {method_name}'] = [modified_graph.number_of_nodes(), modified_graph.number_of_edges()]
    
    
    graph_info_df = pd.DataFrame(graph_info)

    # Combine all results into one DataFrame
    final_results = pd.concat([graph_info_df, combined_results], ignore_index=True)

    # Transpose the DataFrame and set the first row as the header
    final_results = final_results.T
    final_results.columns = final_results.iloc[0]  # Set the first row as the column names
    final_results = final_results.drop(final_results.index[0])  # Drop the first row

    return final_results

In [4]:
#G = create_polarized_graph(1000, 0.2, 0.01)
G= graph_loader("datasets/facebook/facebook_combined.txt")
#G = graph_loader('datasets/congress_network/congress.edgelist')
#color the graph
spectral_bipartition_coloring(G)

Number of Nodes: 7115
Number of Edges: 103689


/Users/quenzer/miniforge3/lib/python3.9/site-packages/sklearn/manifold/_spectral_embedding.py:233: UserWarning: Array is not symmetric, and will be converted to symmetric by average with its transpose.
  adjacency = check_symmetric(adjacency)
/Users/quenzer/miniforge3/lib/python3.9/site-packages/sklearn/manifold/_spectral_embedding.py:259: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


In [5]:
seed = seed_mia(G, 100)
k = 100
max_iter = 1000
budget = 3 * len(seed)
final_results = evaluate_graph_modifications(G, seed, k, max_iter, budget)
final_results

100%|██████████| 1000/1000 [00:20<00:00, 48.29it/s]


IndexError: index 7190 is out of bounds for axis 1 with size 7115

In [ ]:
seed = seed_polarized(G, 100, 1)
k = 15
max_iter = 1000
budget = 3 * len(seed)
final_results = evaluate_graph_modifications(G, seed, k, max_iter, budget)
final_results

100%|██████████| 1000/1000 [00:02<00:00, 359.68it/s]


Metric,Number of Nodes,Number of Edges,Count,Count Standard Deviation,Color Count,Color Count Standard Deviation
Original Graph,475.0,13289.0,239.659,43.514,52.593,20.113
Adapted Graph Adamic Adar,475.0,13576.0,240.221,42.444,52.993,18.493
Adapted Graph PrefAtt,475.0,13589.0,242.834,42.17,56.775,20.088
Adapted Graph Jaccard,475.0,13589.0,256.365,42.901,58.873,19.047
Adapted Graph Degree,475.0,13589.0,250.371,42.496,57.612,20.166
Adapted Graph TopK,475.0,13589.0,237.81,42.684,52.597,18.632
Adapted Graph Prob,475.0,13589.0,273.22,42.868,67.177,20.614
Adapted Graph KKT,475.0,13589.0,238.92,41.224,52.756,18.769
Adapted Graph Random,475.0,13589.0,248.287,42.15,61.953,21.28
Adapted Graph Custom,475.0,13589.0,248.767,41.689,65.017,22.487


In [ ]:
seed = seed_random(G, 100)
k = 15
max_iter = 1000
budget = 3 * len(seed)
final_results = evaluate_graph_modifications(G, seed, k, max_iter, budget)
final_results

100%|██████████| 1000/1000 [00:03<00:00, 328.02it/s]


Metric,Number of Nodes,Number of Edges,Count,Count Standard Deviation,Color Count,Color Count Standard Deviation
Original Graph,475.0,13289.0,273.776,45.852,35.492,16.356
Adapted Graph Adamic Adar,475.0,13573.0,274.596,44.817,37.427,17.088
Adapted Graph PrefAtt,475.0,13589.0,284.094,43.337,50.923,19.775
Adapted Graph Jaccard,475.0,13589.0,310.023,36.47,50.747,15.4
Adapted Graph Degree,475.0,13589.0,276.368,44.806,39.165,17.522
Adapted Graph TopK,475.0,13589.0,276.875,44.896,37.767,17.017
Adapted Graph Prob,475.0,13589.0,304.863,41.458,48.956,17.428
Adapted Graph KKT,475.0,13589.0,275.951,44.733,38.656,17.488
Adapted Graph Random,475.0,13589.0,283.082,42.983,56.341,19.953
Adapted Graph Custom,475.0,13589.0,275.177,45.978,39.682,17.959
